In [1]:
import pandas as pd
import numpy as np
import ast
import random

In [2]:
df = pd.read_parquet("nlg.gzip")

In [3]:
steps = df.directions.tolist()

In [4]:
pairs = []
counter = 0
switch = False
steps_filter_1 = []
sent_unpacked = [j for i in steps[:100000] for j in i]
for recipe in steps[:]:
    for s in range(len(recipe)):
        if "." not in recipe[s]:
            if "!" not in recipe[s]:
                if ":" not in recipe[s]:
                    switch = True
                    break
    if switch:
        switch = False
        counter += 1
    else:
        steps_filter_1.append(recipe)
    # if counter == 100:
    #     break
        # counter += 1
print(counter)
# print(len(steps2))
print(len(steps_filter_1))

164045
2067097


In [5]:
counter = 0
switch = False
steps_filter_2 = []
for r in steps_filter_1[:]:
    for i in r:
        if '[' in i or ']' in i or '\\' in i or '<a' in i or '|' in i:
        # if '<p' in i:
            # print(i)
            switch = True
            break
    if switch:
        counter += 1
        switch = False
    else:
        steps_filter_2.append(r)
counter

2880

In [6]:
random.choice(steps_filter_2)

array(['Boil macaroni until tender.', 'Drain.',
       'Rinse with cold water until cool.',
       'Dice tomatoes, onion and bell pepper.', 'Combine with macaroni.',
       'Add mayonnaise, 3 to 4 tablespoons or enough to blend. Add celery salt and pepper to season to taste.',
       'Chill in refrigerator.'], dtype=object)

In [82]:
pairs = []
sent_unpacked = [j for i in steps_filter_2[:150000] for j in i if len(j.split()) > 1]
steps_filter_3 = [[s for s in r if len(s.split())>1] for r in steps_filter_2[:150000]]
for recipe in steps_filter_3:
    for s in range(len(recipe)):
        if s+1 != len(recipe):
            pairs.append([recipe[s], recipe[s+1]])

In [83]:
pairs = [p for p in pairs if len(p[0])+len(p[1]) <= 400]
len(pairs)

567042

In [84]:
count = 0
pro = []
for i, j in pairs:
    if "." not in i:
        count += 1
        pro.append(i)
    if "." not in j:
        pro.append(j)
        count += 1
print(count)
print(pro)

3583
['Mix together with a fork:', 'The longer it sits, the better it gets!!', 'A good lunch!!', 'Kids love this!', 'Cold pancakes are awful!', 'Cold pancakes are awful!', 'Very good with roast, men love it!', 'High altitude:', 'Mmmmmmm good!', 'Cook until vegetables are tender, but not mushy!', 'Cook until vegetables are tender, but not mushy!', 'Repeat the layering:', 'Repeat the layering:', 'Quick and easy!!', 'It will look like a disaster, but taste great!', 'Preparation time:', 'Preparation time:', 'Do not overcook!', 'The Clergy Pizza special will be a sure way to please the entire family!', 'Enjoy, but be careful, they can be hot!', 'Enjoy, but be careful, they can be hot!', 'Microwave Oven:', 'Microwave Oven:', 'About 40 minutes before serving or early in the day:', 'Calories from fat:', 'Can put glaze on top before baking:', 'Make a well in center and add in order:', 'Make a well in center and add in order:', 'Microwave Directions:', 'Happy eating!', 'Ready to serve!', 'First 

In [85]:
from random import choice
choice(steps)
random.shuffle(pairs)
false_pairs = []
true_pairs = pairs[:len(pairs)//2]
fpair_backup = pairs[len(pairs)//2:]

fpairbank1 = [j for i in fpair_backup for j in i]
fpairbank2 = fpairbank1[:]
random.shuffle(fpairbank2)

fpairbank_pair = [i+"|"+j for i,j in zip(fpairbank1, fpairbank2) if i != j]
true_pairs_joined = set(["|".join(p) for p in true_pairs])
print(len(fpairbank_pair))
# l0, l1 = fpairbank[::1], fpairbank[1::2]
# for i in range(10):
#     print(l0[i], l1[i])
# print(fpairbank[:50])

567000


In [86]:
false_pairs = set()
count = 0
for p in list(set(fpairbank_pair)):
    if not(p in true_pairs_joined) and not(p in false_pairs):
        false_pairs.add(p)

In [87]:
len(false_pairs)

565985

In [88]:
true_pairs_joined = [p for p in true_pairs_joined if len(p) <= 500]
false_pairs = [p for p in false_pairs if len(p) <= 500]
ttag, ftag = [[1,0]]*len(true_pairs_joined), [[0,1]]*len(true_pairs_joined)


In [89]:
pair_df = pd.DataFrame({"sentences":true_pairs_joined+false_pairs[:len(true_pairs_joined)], "tags":ttag+ftag})
pair_df.to_parquet("pair_random.gzip", compression="gzip")

# False pair in same recipe

In [92]:
fpair_in_r = set()
for recipe in steps_filter_3:
    for s in range(len(recipe)):
        temp_sent = random.choice(recipe)
        if recipe[s]!=temp_sent and recipe[s+1:s+2]!=temp_sent:
            temp_pair = recipe[s]+"|"+temp_sent
            if temp_pair not in fpair_in_r and len(temp_pair)<=500:
                fpair_in_r.add(temp_pair)

In [104]:
fpair_in_r = list(fpair_in_r)
random.shuffle(fpair_in_r)
random_tpair_joined = list(true_pairs_joined)[:]
random.shuffle(random_tpair_joined)

In [105]:
plen = len(random_tpair_joined)
ttag2, ftag2 = [[1,0]]*plen, [[0,1]]*plen


In [106]:
pair_inr_df = pd.DataFrame({"sentences":random_tpair_joined+fpair_in_r[:plen], "tags":ttag+ftag})
pair_inr_df.to_parquet("pair_random_inrecipe.gzip", compression="gzip")

In [119]:
# len([i for i in steps_filter_2 if len(" ".join(i).split())>400])/len(steps_filter_2)
from peft import TaskType
for t in TaskType:
    print(t)

TaskType.SEQ_CLS
TaskType.SEQ_2_SEQ_LM
TaskType.CAUSAL_LM
TaskType.TOKEN_CLS
TaskType.QUESTION_ANS
TaskType.FEATURE_EXTRACTION


In [108]:
steps_filter_2

[array(['In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.',
        'Stir over medium heat until mixture bubbles all over top.',
        'Boil and stir 5 minutes more. Take off heat.',
        'Stir in vanilla and cereal; mix well.',
        'Using 2 teaspoons, drop and shape into 30 clusters on wax paper.',
        'Let stand until firm, about 30 minutes.'], dtype=object),
 array(['Place chipped beef on bottom of baking dish.',
        'Place chicken on top of beef.',
        'Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.'],
       dtype=object),
 array(['In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.'],
       dtype=object),
 array(['Boil and debone chicken.',
        'Put bite size pieces in average size square casserole dish.',
        'Pour gravy and cream of mushroom soup over c

In [ ]:
NER = df['NER'].tolist()

In [ ]:
df_test.iloc[0][2]


In [ ]:
NER_unpacked = [j for i in NER for j in i]

In [ ]:
NER_unpacked = [i.lower() for i in NER_unpacked]

In [ ]:
from collections import Counter
count = Counter(NER_unpacked)
df_f = pd.DataFrame.from_dict(count, orient="index").reset_index()
# count

In [ ]:
# df_f = pd.DataFrame({'freq':NER_unpacked})
df_f
# df_f = df_f.rename(columns={0:"freq"}).reset_index()

In [ ]:
import matplotlib.pyplot as plt 
temp_df.freq.plot.density(color='green') 
plt.title('Density plot for freq') 
plt.show()